In [1]:
import sys
import math

In [83]:
path_to_input = "./hmm-training-data/ja_gsd_train_tagged.txt"
sents = []
with open(path_to_input, 'r') as reader:
    for line in reader:
        sents.append(line.strip())

# transition prob - pairwise prob of tag
p_transition = {}
transition_count = {}
# emission prob (#word-in-tag / #vocab-in-tag)
# P(observation|state) or P(word|tag) --> This version is better than a more intuitive version
# P(tag|word) ref. Charniak paper
p_emission = {}
tag_count = {}
word_count = {}


# count transition using sliding window
def split_word_tag_token(token):
    if len(token.split('/')) == 2:
        w = token.split('/')[0]
        t = token.split('/')[1]
    else:
        t = token.split('/')[-1]
        w = ''.join(token.split('/')[:-1])
    return w, t


for sent in sents:
    word_tag = sent.split(' ')
    for wt_token in word_tag:
        w, t = split_word_tag_token(wt_token)
        if t not in tag_count.keys():
            tag_count[t] = 0
        tag_count[t] += 1

        if w not in word_count.keys():
            word_count[w] = {}

        if t not in word_count[w].keys():
            word_count[w][t] = 0

        word_count[w][t] += 1

for sent in sents:
    mod_sent = ('#/#Q0 ' + sent + ' #/#QT').split(' ')
    i = 0
    while i < len(mod_sent) - 1:
        _, t1 = split_word_tag_token(mod_sent[i])
        _, t2 = split_word_tag_token(mod_sent[i + 1])

        if t1 not in transition_count.keys():
            transition_count[t1] = {}

        if t2 not in transition_count[t1].keys():
            transition_count[t1][t2] = 0

        transition_count[t1][t2] += 1
        i += 1
        
tag_list = list(tag_count) + ['#Q0', '#QT']
p_transition = dict.fromkeys(tag_list, {})
p_emission = dict.fromkeys(word_count.keys(), {})

for k in tag_list:
    p_transition[k] = dict.fromkeys(tag_list, 0)
    
for w in word_count.keys():
    p_emission[w] = dict.fromkeys(tag_list, 0)

In [84]:
for t1, next_states in transition_count.items():
    count = 0
    possible_tags = []
    for next_tag, c in next_states.items():
        possible_tags.append(next_tag)
        count += c
        

    for t2 in possible_tags:
        p_transition[t1][t2] = transition_count[t1][t2]
        
    for t2 in p_transition[t1].keys():
        # add one to all transition
        p_transition[t1][t2] += 1
        count += 1
    
    for t2 in p_transition[t1].keys():
        p_transition[t1][t2] /= count

In [99]:
tag_count_for_word_emission = tag_count.copy() # Number of tag --> aka number of emission by tag
sort_tag_count = sorted(tag_count_for_word_emission.items(), key=lambda kv:kv[1], reverse=True)

In [100]:
sort_tag_count

[('NN', 32369),
 ('PS', 18872),
 ('SYM', 17762),
 ('VV', 15128),
 ('AV', 14720),
 ('NNP', 7814),
 ('PN', 7202),
 ('PK', 6403),
 ('PC', 6108),
 ('XV', 5625),
 ('CD', 3874),
 ('XS', 3350),
 ('XSC', 3068),
 ('NB', 2161),
 ('JN', 2020),
 ('NR', 1732),
 ('RB', 1634),
 ('PQ', 1537),
 ('JJ', 1436),
 ('PA', 1063),
 ('JR', 1057),
 ('XP', 993),
 ('NP', 858),
 ('CC', 746),
 ('PH', 557),
 ('PM', 541),
 ('PNB', 467),
 ('XPC', 416),
 ('PE', 257),
 ('PP', 211),
 ('XA', 166),
 ('PF', 147),
 ('AJ', 59),
 ('PX', 42),
 ('UNK', 12),
 ('UH', 9),
 ('AJN', 3)]

In [102]:
# How to deal with unseen word emission ?
# Pick top N tags and then add as unseen
# HMM Constraint: Total Probability of state qi emitting the observation vj must be 1
word_count["##unseen##"] = {}
p_emission['##unseen##'] = dict.fromkeys(tag_list, 0)

pick_top_n = int(len(sort_tag_count)/4)
total_top_n = sum([v for k, v in sort_tag_count[0:pick_top_n]])

for i in range(0, pick_top_n):
    tag, count = sort_tag_count[i]
    distribution = int(pick_top_n * (count / total_top_n))
    word_count['##unseen##'][tag] = distribution
    tag_count_for_word_emission[tag] += distribution
    
for w, poss_tag_count in word_count.items():
    for t, n in poss_tag_count.items():
        p_emission[w][t] = n / tag_count_for_word_emission[t]

In [103]:
p_emission['##unseen##'] 

{'NN': 6.178369528281487e-05,
 'PS': 5.298574683410163e-05,
 'PK': 0.0,
 'NNP': 0.0,
 'PN': 0.0,
 'VV': 6.609822195782933e-05,
 'PC': 0.0,
 'SYM': 5.629679671226707e-05,
 'NB': 0,
 'JN': 0,
 'XV': 0,
 'JR': 0,
 'CD': 0,
 'XSC': 0,
 'AV': 6.793016778751444e-05,
 'CC': 0,
 'XP': 0,
 'XS': 0,
 'PH': 0,
 'NP': 0,
 'PE': 0,
 'PA': 0,
 'XPC': 0,
 'PP': 0,
 'XA': 0,
 'PQ': 0,
 'NR': 0,
 'RB': 0,
 'JJ': 0,
 'PM': 0,
 'PNB': 0,
 'PF': 0,
 'AJ': 0,
 'PX': 0,
 'UH': 0,
 'UNK': 0,
 'AJN': 0,
 '#Q0': 0,
 '#QT': 0}

In [89]:
sorted_top_tag_from_q0 = sorted(transition_count['#Q0'].items(), key = lambda kv: kv[1], reverse=True)
sorted_top_tag_from_q0

[('NN', 2722),
 ('NNP', 1102),
 ('CD', 577),
 ('CC', 557),
 ('JR', 382),
 ('RB', 361),
 ('NR', 339),
 ('NP', 332),
 ('VV', 173),
 ('JN', 168),
 ('SYM', 156),
 ('XP', 146),
 ('XPC', 59),
 ('JJ', 55),
 ('UH', 3),
 ('UNK', 1)]

In [71]:
ntags = sum([v for k,v in tag_count.items()])
l_tags = [(k,v/ntags) for k,v in tag_count.items()]
l_tags
# l_tags = sorted(l_tags,key=lambda x: x[1], reverse=True)
# p_emission['##unseen##'] = {}
# for t,prob in l_tags:
#     p_emission['##unseen##'][t] = prob
# p_emission['##unseen##']

[('SP', 0.04952557613787457),
 ('FS', 0.04320354758186937),
 ('S', 0.1991710715566682),
 ('E', 0.15177578355113236),
 ('RD', 0.12766150156329817),
 ('A', 0.06285436872099384),
 ('FC', 0.004496067299714875),
 ('RI', 0.015520670678467787),
 ('VA', 0.021458667700411927),
 ('VM', 0.006369126763012691),
 ('V', 0.09622525985529981),
 ('PR', 0.01043044138265844),
 ('FB', 0.019694296407131395),
 ('PI', 0.003438169111546669),
 ('FF', 0.045982341795311195),
 ('CC', 0.027349566515348582),
 ('N', 0.017371992507762148),
 ('PC', 0.015698194689496014),
 ('AP', 0.0061626192399798566),
 ('NO', 0.0035359884645622223),
 ('B', 0.03152319224401219),
 ('I', 0.00022462221803571492),
 ('T', 0.001373093881218322),
 ('PE', 0.0023440415333727026),
 ('CS', 0.010263786188631942),
 ('DI', 0.005441654378865223),
 ('BN', 0.006619109554052438),
 ('PD', 0.0027896630304435565),
 ('DD', 0.003673660146584112),
 ('DE', 1.449175600230419e-05),
 ('DQ', 0.0031700716255040414),
 ('PQ', 0.002981678797474087),
 ('SW', 0.00077168

In [104]:
word_count

{'ホッケー': {'NN': 1},
 'に': {'PS': 4728, 'XV': 520, 'PX': 30, 'PC': 14},
 'は': {'PK': 4830},
 'デンジャラスプレー': {'NNP': 1},
 'の': {'PN': 7202, 'PM': 540, 'PNB': 426, 'XV': 55, 'PE': 1},
 '反則': {'NN': 2},
 'が': {'PS': 3607, 'PC': 648},
 'ある': {'VV': 455, 'XA': 86, 'AV': 7},
 'ので': {'PC': 227},
 '、': {'SYM': 6184},
 '膝': {'NN': 2},
 'より': {'PS': 90, 'VV': 17, 'RB': 1},
 '上': {'NN': 10, 'XS': 44, 'NB': 23, 'XP': 4},
 'ボール': {'NN': 11},
 'を': {'PS': 4720},
 '浮かす': {'VV': 1},
 'こと': {'NB': 727, 'NN': 31, 'VV': 2},
 '基本的': {'JN': 15},
 'なる': {'VV': 169, 'AV': 51, 'XV': 3},
 'その': {'JR': 337},
 '例外': {'NN': 6},
 '一': {'CD': 134, 'XS': 9, 'NN': 3, 'XP': 11},
 'つ': {'XSC': 96, 'VV': 19},
 'この': {'JR': 353},
 'スクープ': {'NN': 1},
 'である': {'AV': 399},
 '。': {'SYM': 6929},
 'また': {'CC': 222, 'RB': 7},
 '行き': {'VV': 24, 'AV': 5, 'XS': 1},
 'たい': {'AV': 68, 'VV': 1},
 'そんな': {'JR': 21},
 '気持ち': {'NN': 10},
 'さ': {'VV': 87, 'XV': 988, 'NB': 39, 'AV': 9, 'PE': 3},
 'せ': {'AV': 117, 'XV': 17, 'VV': 3},
 'て': {'

In [2]:
import sys
import json
import math

In [6]:
def split_word_tag_token(token):
    if len(token.split('/')) == 2:
        w = token.split('/')[0]
        t = token.split('/')[1]
    else:
        t = token.split('/')[-1]
        w = ''.join(token.split('/')[:-1])
    return w, t

In [4]:
path_to_input = "./hmm-training-data/it_isdt_dev_raw.txt"
with open('./hmmmodel.txt', 'r') as reader:
    hmm_model = json.load(reader)

sents = []
out_setns = []
with open(path_to_input, 'r') as reader:
    for line in reader:
        sents.append(line.strip())

In [5]:
sent = "ただし 、 50 周年 ソング に 変更 後 は 、 ED も 歌 つき の もの が 使わ れ た 。"

In [6]:
def handle_unseen(T, idx, b):
    if T[idx] not in b.keys():
        b[T[idx]] = b['##unseen##']

In [7]:
# Viterbi
back_pointer = {}
T = sent.split(' ')
Q = hmm_model['tag']
a = hmm_model['p_transition']
b = hmm_model['p_emission']
q0 = '#Q0'
qt = '#QT'
num_q = len(Q)
num_t = len(T)
# Initialize by 1 which is INF here and will be skip for unreachable state
prob = [[2 for t in range(num_t + 1)] for q in range(num_q)] # num_t + 1 include last state QT

# Note:
# cal neg value --> find max arg --> skip no value
# or negate to get pos value -> find min --> like find min entropy

# TODO - Smoothing a[q'][q] == 0
# initialize at step t_0
for iq, q in enumerate(Q):
    back_pointer[q] = {} # initialize backpointer with all key 'q' in Q
    # TODO - handle unseen word b[T[0]]
    handle_unseen(T, 0, b)
    if a[q0][q] == 0 or b[T[0]][q] == 0:
        continue
    prob[iq][0] = math.log(a[q0][q]) + math.log(b[T[0]][q])
    back_pointer[q][0] = q0
    # Handle no transition

# Run from step t_1 to t_T
for t in range(1, num_t):
    # find argmax prob
    argmax_b = 1
    has_transition = False
    for iq, q in enumerate(Q):
        for ipq, pq in enumerate(Q): # Enumerate every q in previous t 
            handle_unseen(T,t,b)
            if prob[ipq][t-1] == 2 or a[pq][q] == 0 or b[T[t]][q] == 0:
                continue
            if prob[iq][t] == 2:
                prob[iq][t] = prob[ipq][t-1] + math.log(a[pq][q]) + math.log(b[T[t]][q])
                argmax_b = prob[ipq][t-1] + math.log(a[pq][q])
                back_pointer[q][t] = pq
                if t == 23:
                    print("Assign q:{} t:{} to pq:{}".format(q,t,pq))
                has_transition = True
            else:
                prob[iq][t] = max(prob[iq][t], prob[ipq][t-1] + math.log(a[pq][q]) + math.log(b[T[t]][q]))
                if prob[ipq][t-1] + math.log(a[pq][q]) > argmax_b:
                    argmax_b = prob[ipq][t-1] + math.log(a[pq][q])
                    back_pointer[q][t] = pq
                    if t == 23:
                        print("Assign q:{} t:{} to pq:{}".format(q,t,pq))
                    
    if not has_transition:
        print("No Transition at {}".format(T[t]))
        # Prob: when state 't' is unreachable
        # case: there is only one transition to QT and that word has no b[w][pos] value
        # Or the only b[w][pos] we have is not reachable
        # Sol: use most likely tag for current 't' and use most likely tag for prev t
        max_prev_t = 2
        most_likely_pq = None
        for ipq, pq in enumerate(Q):
            if prob[ipq][t-1] == 2:
                continue
            if max_prev_t == 2:
                max_prev_t = prob[ipq][t-1]
                most_likely_pq = pq
            else:
                if prob[ipq][t-1] > max_prev_t:
                    max_prev_t = prob[ipq][t-1]
                    most_likely_pq = pq
         
        max_cur_emission = 2
        cq = None
        most_liekly_iq = -1
        for iq, q in enumerate(Q):
            if b[T[t]][q] == 0:
                continue
            if max_cur_emission == 2:
                max_cur_emission = math.log(b[T[t]][q])
                cq = q
                most_liekly_iq = iq
            else:
                if math.log(b[T[t]][q]) > max_cur_emission:
                    max_cur_emission = math.log(b[T[t]][q])
                    cq = q
                    most_liekly_iq = iq
        back_pointer[cq][t] = most_likely_pq
        prob[most_liekly_iq][t] = max_prev_t + max_cur_emission
        print("Put transition from {}/{} to previous state {}/{}".format(T[t],cq,T[t-1], most_likely_pq))
        print(back_pointer[cq])

# At termination step: QT
bpt_qt = None
argmax_qt = 2
for iq, q in enumerate(Q):
    # TODO - add smoothing when a[q][qt] == 0
    # Edge case: there is only one transition to QT and that word has no a[q][qt] value
    if prob[iq][num_t-1] == 2 or a[q][qt] == 0:
        continue
    if argmax_qt == 2:
        argmax_qt = prob[iq][num_t-1] + math.log(a[q][qt])
        bpt_qt = q
    else:
        if prob[iq][num_t-1] + math.log(a[q][qt]) > prob[iq][num_t]:
            prob[iq][num_t] = max(prob[iq][num_t], math.log(a[q][qt]))
            bpt_qt = q

# Return backtrack path
q = bpt_qt
sent_tag = []
t = len(T)-1
while q != '#Q0' and t >= 0:
    print("Word: {} Q: {}, t: {}".format(T[t],q, t))
    sent_tag.append(q)
    q = back_pointer[q][t]
    t -= 1
sent_tag.reverse()
sent_tag

Word: 。 Q: SYM, t: 20
Word: た Q: AV, t: 19
Word: れ Q: AV, t: 18
Word: 使わ Q: VV, t: 17
Word: が Q: PS, t: 16
Word: もの Q: NB, t: 15
Word: の Q: PN, t: 14
Word: つき Q: XS, t: 13
Word: 歌 Q: NN, t: 12
Word: も Q: PK, t: 11
Word: ED Q: NN, t: 10
Word: 、 Q: SYM, t: 9
Word: は Q: PK, t: 8
Word: 後 Q: NB, t: 7
Word: 変更 Q: VV, t: 6
Word: に Q: PS, t: 5
Word: ソング Q: NN, t: 4
Word: 周年 Q: XSC, t: 3
Word: 50 Q: CD, t: 2
Word: 、 Q: SYM, t: 1
Word: ただし Q: CC, t: 0


['CC',
 'SYM',
 'CD',
 'XSC',
 'NN',
 'PS',
 'VV',
 'NB',
 'PK',
 'SYM',
 'NN',
 'PK',
 'NN',
 'XS',
 'PN',
 'NB',
 'PS',
 'VV',
 'AV',
 'AV',
 'SYM']

In [8]:
output = ""
for i,token in enumerate(T):
    output += "{}/{} ".format(token, sent_tag[i])
output = output.strip()
output

'ただし/CC 、/SYM 50/CD 周年/XSC ソング/NN に/PS 変更/VV 後/NB は/PK 、/SYM ED/NN も/PK 歌/NN つき/XS の/PN もの/NB が/PS 使わ/VV れ/AV た/AV 。/SYM'

In [ ]:
# Grading
import sys

def split_word_tag_token(token):
    if len(token.split('/')) == 2:
        w = token.split('/')[0]
        t = token.split('/')[1]
    else:
        t = token.split('/')[-1]
        w = ''.join(token.split('/')[:-1])
    return w, t


path_to_result = './hmmoutput.txt'
path_to_dev = './hmm-training-data/it_isdt_dev_tagged.txt'
hmm_output = []
devset = []
with open(path_to_result, 'r') as reader:
    for line in reader:
        hmm_output.append(line)
        
with open(path_to_dev, 'r') as reader:
    for line in reader:
        devset.append(line)

if len(hmm_output) != len(devset):
    raise Exception("Number of lines is not equal. Exit with Error(1)")

ntags = 0
ncorrect = 0
for i in range(0,len(devset)):
    lhmm = hmm_output[i].split(' ')
    ldev = devset[i].split(' ')
    for j in range(0, len(ldev)):
        hmm_w, hmm_t = split_word_tag_token(lhmm[j])
        dw, dt = split_word_tag_token(ldev[j])
        if hmm_w != dw:
            raise Exception("Word does not match hmm_w={} dw={}. Exit with Error(1)".format(hmm_w, dw))
        if hmm_t == dt:
            ncorrect += 1
        ntags += 1

print("Accuracy: {}%".format(ncorrect*100/ntags))